In [1]:
import sys
sys.path.insert(1, '/mnt/f/Dev/LMEmbeddingAnalysis/') # Change to what you need

In [2]:
from models.CustomWordEmbeddings import CustomWordEmbeddings
from utils.utils import get_model, get_saved_embedding, clear_cuda_memory
from utils.mteb_utils import mteb_meta, test_all_mteb
import torch
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Embedding Analysis

Uncomment as needed: 

In [3]:
# model_names = [
#                 'pythia-14m',
#                 'pythia-31m',
#                 'pythia-70m', 
#                 'pythia-160m', 
#                 'pythia-410m', 
#                 'pythia-1b', 
#                 'pythia-1.4b', 
#                 'pythia-2.8b', 
#                 'pythia-6.9b', 
#                 'pythia-12b',
#                 ] 
# tokenizer_name = 'EleutherAI/gpt-neox-20b'

# model_names = [
#     "opt-125m",
#     "opt-350m",
#     "opt-1.3b",
#     "opt-2.7b",
#     "opt-6.7b",
#     "opt-13b",
#     "opt-30b",
#     "opt-66b",
#     "opt-175b",
# ]
# tokenizer_name = "facebook/opt-350m"

# model_names = [
#     "Cerebras-GPT-111M",
#     "Cerebras-GPT-256M",
#     "Cerebras-GPT-590M",
#     "Cerebras-GPT-1.3B",
#     "Cerebras-GPT-2.7B",
#     "Cerebras-GPT-6.7B",
#     "Cerebras-GPT-13B",
# ]
# tokenizer_name = "cerebras/Cerebras-GPT-111M"

# model_names = [
#     "bloom-560m",
#     "bloom-1b1",
#     "bloom-1b7",
#     "bloom-3b",
#     "bloom-7b1",
#     "bloom",
# ]
# tokenizer_name = "bigscience/bloom"

model_names = [
    "t5-v1_1-small",
    # "t5-v1_1-base",
    # "t5-v1_1-large",
    # "t5-v1_1-xl",
    # "t5-v1_1-xxl",
]
tokenizer_name = "google/t5-v1_1-small"

embedding_types = ["random_xn", "full", "crop_128", "pad_14336"]

pooling_modes = ["mean"] #, "max", "cls"] #Note, I had issues with these so never used them

for pooling_mode in pooling_modes: 
    for embedding_type in embedding_types: 
        for model_name in model_names:
            model, output_folder = get_model(model_name, 
                                            embedding_type=embedding_type,
                                            pooling_mode=pooling_mode,
                                            tokenizer_name=tokenizer_name,
                                            save_model=False,
                                            )
            if os.path.isfile(output_folder + "mteb_metadata.md"):
                print("Skipping!")
                continue
            print("NOW EVALUATING MODEL", model, output_folder)

            test_all_mteb(model, output_folder, verbosity=0)
            clear_cuda_memory(model)
            mteb_meta(output_folder, output_folder)

512 EMBED DIM
Skipping!
512 EMBED DIM
Skipping!
512 EMBED DIM
Skipping!
512 EMBED DIM
NOW EVALUATING MODEL SentenceTransformer(
  (0): CustomWordEmbeddings(
    (emb_layer): Embedding(32128, 14336)
  )
  (1): Pooling({'word_embedding_dimension': 512, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) /mnt/f/Dev/LMEmbeddingAnalysis/mteb_analyses/t5-v1_1-small_pad_14336_mean/
Running task: SCIDOCS


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, beir, s2p

  0%|          | 0/25657 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Batches:   0%|          | 0/3208 [00:00<?, ?it/s]

## Analyis for The Random Baseline

In [3]:
model_tok_rand_sizes = [  128,   256,   512,   768,  1024,  1088,  1536,  2048,  2560,  4096,  5120,  7168, 9216, 12288, 14336]

tokenizers = [
                "bigscience/tokenizer",
                "cerebras/Cerebras-GPT-111M",
                "facebook/opt-350m",
                "google/t5-v1_1-small",
                'EleutherAI/gpt-neox-20b']

tokenizer_nicknames = [
    "Bloom",
    "Cerebras",
    "OPT",
    "T5",
    "GPT-NeoX"
]

tokenizer_embedding_sizes = {
    "bigscience/tokenizer":250880,
    "cerebras/Cerebras-GPT-111M":50257,
    "facebook/opt-350m":50272,
    "google/t5-v1_1-small":32128,
    'EleutherAI/gpt-neox-20b':50304,
}

embedding_types = ["random_xn_tok"]

pooling_modes = ["mean"] #, "max", "cls"] #Note, I had issues with these so never used them

for pooling_mode in pooling_modes: 
    for embedding_type in embedding_types: 
        for tokenizer_name, tok_nick in zip(tokenizers, tokenizer_nicknames):
            model_base_name = "random_tok_" + tok_nick + "_"
            for model_size in model_tok_rand_sizes:
                model_name = model_base_name + str(model_size)
                embedding_size = (tokenizer_embedding_sizes[tokenizer_name], model_size)
                model, output_folder = get_model(model_name, 
                                                embedding_type=embedding_type,
                                                pooling_mode=pooling_mode,
                                                tokenizer_name=tokenizer_name,
                                                embedding_size=embedding_size,
                                                save_model=False)
                if os.path.isfile(output_folder + "mteb_metadata.md"):
                    print("Skipping!")
                    continue
                print("NOW EVALUATING MODEL", model, output_folder)

                test_all_mteb(model, output_folder, verbosity=0)
                clear_cuda_memory(model)
                mteb_meta(output_folder, output_folder)

128 EMBED DIM
128 EMBED DIM
NOW EVALUATING MODEL SentenceTransformer(
  (0): CustomWordEmbeddings(
    (emb_layer): Embedding(250880, 128)
  )
  (1): Pooling({'word_embedding_dimension': 128, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) /mnt/f/Dev/LMEmbeddingAnalysis/mteb_analyses/random_tok_Bloom_128_random_xn_tok_mean/
Running task: SCIDOCS


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- SCIDOCS, beir, s2p

  0%|          | 0/25657 [00:00<?, ?it/s]

...

  0%|          | 0/25657 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Batches:   0%|          | 0/3208 [00:00<?, ?it/s]

2048 EMBED DIM
